# Data Measurement

**Tian Lou** \
Ohio Education Research Center \
The Ohio State University

**Xiangyu Ren** \
New York University

**Anna-Carolina Haensch** \
University of Maryland \
LMU Munich

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.10256974.svg)](https://doi.org/10.5281/zenodo.10256974)

**This notebook is developed for the [Data Literacy and Evidence Building Executive Class](https://www.socialdatascience.umd.edu/data-literacy).**

**The "Syntucky" data, which is synthetic in nature, is exclusively designed for training exercises. It is not intended to derive meaningful insights or make determinations about real-world populations.**

## Goals:
Our overarching goal in this series of notebooks is **to examine job quality of the 2015 Syntucky cohort**. In the Data Exploration notebook, we learned how to import the cohort data, how to subset the cohort data by major and degree levels, and how to generate descriptive statistics for outcome variables, such as year 7 earnings. In this notebook, we will first **identify three groups of students: completers, non-completers, and degree-pursuers**. Then we will **develop job quality measurements** in order to compare labor market outcomes for students in these three groups. 

**The specific questions we seek to answer in this notebook are**:
1. What percentages of the 2015 cohort are completers, non-completers, and degree pursuers? 
2. What measures can we use to examine year 7 job quality for completers, non-completers, and degree pursuers?
3. Which group have higher job quality in year 7, completers, non-completers, or degree pursuers?
4. How does job quality vary by major for completers, non-completers, and degree pursuers?

**After completing this notebook, you should:**
1. Be able to identify different groups of students based on their degrees obtained and college enrollment status.
2. Become familar with various job quality measures and be able to design measures based on your research needs.
3. Learn how to measure job quality for different groups of students and students from different majors.

## 1. Import Data

In [ ]:
#Load libraries
import pandas as pd
import numpy as np

Let's import the 2015 Syntucky cohort data first. Before running the code below, please change <font color='red'> **YOUR DATA DIRECTORY**</font> to your own file path.

In [ ]:
#Define data folder directory
data_directory = 'YOUR DATA DIRECTORY'

#Read in 2015 cohort data
df_2015 = pd.read_csv(data_directory+'syntucky_cohort_2015.csv')

#Check the first five rows of the data
df_2015.head()

## 2. Conduct Analysis

### 2.1. Identify Completers, Non-Completers, and Degree Pursuers

Our first goal is to identify three mutually exclusive groups: completers, non-completers, and degree pursuers. In this analysis, we will focus on students' outcomes at the sixth and seventh years after college entrance. If a student earned at least an associate's degree within six years after college entrance, we define him/her as a **completer**. If a student did not earn any degree, we further examine his/her college enrollment status in year 7. If the student did not enroll in college in year 7, we define him/her as a **non-completer**. If the student enrolled in college in year 7, we define him/her as a **degree-pursuer**.

To generate these three groups of students from the cohort data, we need two key variables: **high_completion_label** and **year7_enrolled**. Recall that `high_completion_label` is the highest degree a person earned within six years after college entrance and `year7_enrolled` indicates whether a student enrolled in college in year 7. By using these two variables, we can formally define completers, non-completers, and degree-pursuers as below:

- **Completers**: Students who have earned degrees within six years since college entrance, i.e., `high_completion_label` is at least *Associate*.

- **Non-Completers**: Students who did NOT enroll in college in year 7 and have not yet earned their degrees, i.e., `year7_enrolled == 0` and `high_completion_label` is NOT at least *Associate*.

- **Degree-Pursuers**: Students who enrolled in college in year 7 and have not yet earned their degrees, ,i.e., `year7_enrolled == 1` and `high_completion_label` is NOT at least *Associate*.

Before we jump into the analysis, we should investigate the unique values in *high_completion_label*. In the code below, we use the `.groupby()` and `.agg()` functions to check the unique values in *high_completion_label* and the number of students by degree level. Note that we also include `dropna = False` in the `.groupby()` function so that we can check the number of rows with null values in *high_completion_label*.

In [ ]:
# Check `high_completion_label` unique values 
# and the number of students by highest degree level
df_2015.groupby(['high_completion_label'], dropna = False)['id'].agg(['count']).reset_index()

We can see that the largest category is students without a degree (`NaN`). Among students who earned degrees, most of their highest degrees are associate or bachelor. The other degree levels in *high_completion_label* are certificates, diploma, master, and even doctoral degrees. We do not consider certificate and diploma as formal college degrees and therefore will not count students who only earned certificates and diploma as completers. Moreover, while it's not impossible for some students to obtain a doctoral degree within six years after college entrance, we suspect these data points are more likely to be data errors and will not consider them in the later analysis. In the code below, we remove the records where *high_completion_label* is *Doctoral* from `df_2015`. The `!=` operator means "is not equal to".

In [ ]:
#Remove the records where high_completion_label is Doctoral
df_2015 = df_2015[df_2015['high_completion_label'] != 'Doctoral']

Now, we can use the function `np.select()` to generate an indicator, `group`, in `df_2015`. To use `np.select()`, you need to define: 1) a list of conditions based on variables in `df_2015`; and 2) a list of values (or choices) for each condition. In the code below, we first define five conditions: 1) completers whose highest degrees are associate; 2) completers whose highest degrees are bachelor; 3) completers whose highest degrees are master; 4) students who did not enroll in college in year 7 and did not earn at least an associate degree, i.e., non-completers; 5) students who enrolled in college in year 7 and did not earn at least an associate degree, i.e., degree pursuers.

> In the non-completers and degree pursuers conditions, we use the `.isin()` function and `~` (read as tilde), to identify students who did not earn an *Associate*, *Bachelor*, or *Master* degree. The `isin()` function identifies rows where a particular column's values match any of the specified values in a given list (for example, the list ['Associate', 'Bachelor', 'Master']). `~` is a bitwise NOT operator. When being used with a Pandas DataFrame, it negates the result of a condition, effectively flipping True values to False and False values to True. 

In the choices list, we define five values that will be assigned to the `group` column based on each condition. For example, if a student's `high_completion_label` value is *Associate*, his/her `group` value will be *completer, Associate*.

In the last line of code, we use the `np.select()` function, the `conditions` list, and the `choices` list to generate the new column `group` in `df_2015`. We assign the default value of `group` to be `np.NaN`. This means that if any row does not satisfy any of the five conditions we list, the value of `group` for that row will be null value.

In [ ]:
#Generate a group indicator

#Conditions list
conditions = [df_2015['high_completion_label'] == 'Associate', #Completers whose highest degrees are associate
              df_2015['high_completion_label'] == 'Bachelor', #Completers whose highest degrees are bachelor
              df_2015['high_completion_label'] == 'Master', #Completers whose highest degrees are master
              ((df_2015['year7_enrolled'] == 0) & 
               ( ~ df_2015['high_completion_label'].isin(['Associate', 'Bachelor', 'Master']))), #Non-completers
             ((df_2015['year7_enrolled'] == 1) & 
               ( ~ df_2015['high_completion_label'].isin(['Associate', 'Bachelor', 'Master'])))] #Degree pursuers

#Choices (or values) list
choices = ['completer, Associate', 
           'completer, Bachelor', 
           'completer, Master', 
           'non-completer', 
           'degree pursuer']

#Assign results to the indicator 'group' based on conditions; Default choice is the null value
df_2015['group'] = np.select(conditions, choices, default = np.NaN)

We can use `.groupby()` and `.agg()` functions again to generate number of students by the five groups we just generated. We can save the results to a DataFrame `df_cnt_group` and also calculate the percentage of students in each group.

In [ ]:
#Counts of students in each group; save the result to DataFrame 'df_cnt_group'
df_cnt_group = df_2015.groupby(['group'], dropna = False)['id'].agg(['count']).reset_index()

#Add a new column, 'percent'
#Recall that df_2015.shape[0] is the number of rows in 'df_2015', i.e., number of students in 'df_2015'
df_cnt_group['percent'] = round(df_cnt_group['count']/df_2015.shape[0], 2)

#See the results
df_cnt_group

#### **Checkpoint 1: Generate Completers, Non-Completers, and Degree Pursuers**

1. Load the 2013 cohort data and save it in `df_2013`.
2. Add the `group` column to the 2013 cohort data. The `group` column identifies *'completer, Associate'*, *'completer, Bachelor'*, *'completer, Master'*, *'non-completer'*, and *'degree pursuer'*.
3. What percentages of the 2013 cohort are completers, non-completers, and degree pursuers?

### 2.2. Basic Job Quality Measures

Our second goal is to analyze job quality for completers, non-completers, and degree pursuers during the seventh year after college entrance. There are multiple ways to measure job quality. Which measures are the most appropriate depends on the specific questions you are seeking to answer. Let's start with two basic measurements: *earnings* and *number of jobs*.

- **Earnings**: Earnings are one of the most direct job quality measure. In the 2015 cohort data, we have year 5 to year 7 inflation adjusted yearly earnings, i.e., *year5_earnings*, *year6_earnings*, and *year7_earnings*. Note that this measure only considers people who had positive in-state earnings in year 7.

- **Number of Jobs**: The number of jobs an individual holds can be another indicator for job quality. Having to work multiple jobs may suggest that a single job does not provide sufficient income or hours, implying lower job quality. In the 2015 cohort data, we also have the number of distinct employers a person worked for during a year, i.e., *year5_ct_employers*, *year6_ct_employers*, and *year7_ct_employers*.

Let's compare the average year 7 earnings and number of employers between completers, non-completers, and degree-pursuers. We can see that completers on average have higher earnings and fewer employers than non-completers and degree pursuers in year 7. Within the completer group, the higher the degree level, the higher the average earnings and the fewer employers. 

In [ ]:
#Average year 7 earnings and number of employers by group
#We put the variable list vertically to improve the readability of the code
df_job_quality_group = df_2015.groupby(['group'])[['year7_earnings', 
                                                   'year7_ct_employers']].agg(['mean']).reset_index()

#See the result
df_job_quality_group

#### **Checkpoint 2: Generate Basic Job Quality Measure**

Please calculate year 7 average earnings, and average number of employers for completers, non-completers, and degree pursuers in the 2013 cohort. Do you observe the same trends as the 2015 cohort?

### 2.3. Additional Job Quality Measurement

Administrative data allows us to construct additional job quality measures that go beyond simple average earnings and number of employers. For example, in the previous analysis, we consider earnings from any job, even if someone only earned $100 from that job. However, we can put additional restrictions on the standard of high-quality jobs. In this section, we examine job quality by using three additional measures:

- **High-Earnings Job**: Jobs that pay above a certain threshold. For example, yearly earnings from a full-time federal minimum wage job is $15,080 (7.25 dollars per hour, 2080 hours per year). We can define jobs that pay above this threshold as high-earnings jobs. However, you may raise the threshold based on other criteria, such as above living wages.

- **Employment Duration**: Number of quarters a person was employed during a year. We can find this information in variables *year5_ct_qtrs_employed*, *year6_ct_qtrs_employed*, and *year7_ct_qtrs_employed*.

- **Average Earnings per Employed Quarter**: We can combine earnings and employment duration to capture both the regularity of employment and the earnings associated with it. This measure offers a more comprehensive view of the job landscape.

Since we already have the employment duration measure in the data, we only need to construct the high earnings job indicator and calculate average earnings per employed quarter. In the code below, first, we generate a dummy variable `year7_high_earnings`, in which if a person earned above 15,080 dollars in year 7, its value is one. Otherwise, its value is zero. Note that the condition `df_2015['year7_earnings'] > 15080` only returns `True` or `False`, depending on the values in `year7_earnings`. In Python, we can multipy the condition by 1 and Python converts the `True`/`False` values to `1`/`0`.  To calculate average earnings per employed quarter, we just need to divide total earnings in year 7, `df_2015['year7_earnings']`, by employment duration in year 7, `df_2015['year7_ct_qtrs_employed']`.

> **You can experiment with different thresholds for high-earnings jobs and see how the results change.** You just need to change the number `15080` in the first line of the code below.

In [ ]:
#High earnings job indicator
df_2015['year7_high_earnings'] = (df_2015['year7_earnings'] > 15080) * 1

#Average earnings per employed quarter
df_2015['year7_avg_qtr_earnings'] = df_2015['year7_earnings'] / df_2015['year7_ct_qtrs_employed']

Now we can examine job quality for completers, non-completers, and degree pursuers by using all five job quality measures. We can see that on average, completers were more likely to have high earnings jobs, were employed for slightly more quaters, and had higher earnings per employed quarter in year 7 than non-completers and degree pursuers. One interesting finding is that while bachelor's and master's degrees holders had higher earnings, fewer employers, and higher average earnings per employed quarter than associate's degree holders, they were less likely to have high-earnings jobs and worked slightly fewer quarters than associate's degree holders in year 7.

> You can add median earnings by replacing `.agg(['mean'])` with `agg(['mean', 'median'])`.

In [ ]:
#Define the list of job quality measures
#This step just puts all column names in one list to improve the readability of the groupby() code.
measure_list = ['year7_earnings', 'year7_ct_employers', 'year7_high_earnings', 
                'year7_ct_qtrs_employed', 'year7_avg_qtr_earnings']

#Job Quality by group
df_job_quality_group = df_2015.groupby(['group'])[measure_list].agg(['mean']).reset_index()

#See the result
df_job_quality_group

**To ensure that your results are meaningful, it is important to check the missingness of the key variables.** In the code below, we check the missingness of `year7_earnings`. First, we generate an indicator, `y7_earn_missing`, which shows whether a person's year 7 earnings are null value. Second, we use the `.groupby()` and `.agg()` functions to calculate missing rate and number of people with missing year 7 earnings for completers, non-completers, and degree pursuers. Finally, we can use the `.rename()` function to change column names so that it's easier for us to read the results.

In [ ]:
#Check missingness of 'year7_earnings' by completer, non-completer, and degree pursuer

#Generate year 7 earnings missing indicator
df_2015['y7_earn_missing'] = (df_2015['year7_earnings'].isna() == True) * 1

#Calculate missing rate and number of people with missing year 7 earnings by group
df_missing = df_2015.groupby(['group'])['y7_earn_missing'].agg(['mean', 'sum']).reset_index()

#Rename column names
df_missing = df_missing.rename(columns = {'mean' : 'year7_earn_missing_rate',
                                          'sum' : 'year7_earn_missing_cnt'})

df_missing

#### **Checkpoint 3: Generate Additional Job Quality Measures**

1. Please calculate year 7 high-earnings job rate, average employment duration, and average earnings per employed quarter for completers, non-completers, and degree pursuers in the 2013 cohort. Do you observe the same trends as the 2015 cohort?

2. There are other measures available in the data, such as:
   -  `year7_max_qrts_one_employer`: Number of quarters a person employed by the most consistent employer in year 7.
   -  `year7_earnings_most_consistent_employer`: inflation-adjusted earnings from the most consistent employer in year 7.

    See if you can use them to create other job quality measures.

3. Check the missingness of your outcome variables.

### 2.4. Job Quality by Major

Finally, we are also interested in job quality by major and group. For example, does a graduate with a bachelor's degree in computer science earn more than a graduate with a bachelor's degree in education? For another example, if two students were both in computer science major, but one has a bachelor's degree and the other one has an associate's degree, which of them would have higher earnings in year 7?

To answer these questions, for completers, we need to use `high_completion`, which includes the major associated with the highest degree a person earned. However, non-completers and degree pursuers do not have `high_completion` majors, since they did not earn a degree yet. In this case, we can use their first enrolled major, `first_enroll`.

In the code below, we separate the completers, non-completers, and degree pursuers into two DataFrames. This way, it is easier for us to use the `groupby()` and `agg()` methods to generate descriptive statistics later. The operator `|` means `or` in Python.

In [ ]:
#Save completers to DataFrame df_2015_comp
df_2015_comp = df_2015[(df_2015['group'] == 'completer, Associate') | 
                       (df_2015['group'] == 'completer, Bachelor') |
                       (df_2015['group'] == 'completer, Master')]

#Save non-completers and degree pursuers to df_2015_non_comp
df_2015_non_comp = df_2015[(df_2015['group'] == 'non-completer') | 
                           (df_2015['group'] == 'degree pursuer')]

Now, we can generate job quality summary statistics for completers, non-completers, and degree pursuers. *Note that the last measure is the year 7 earnings missing indicator we created in the previous section.* You should use the missing rates to decide if and how your results are biased.

In [ ]:
#Define the list of job quality measures
#This step just puts all column names in one list to improve the readability of the groupby() code.
measure_list = ['year7_earnings', 'year7_ct_employers', 'year7_high_earnings', 
                'year7_ct_qtrs_employed', 'year7_avg_qtr_earnings', 'y7_earn_missing']

#Job Quality by major for completers
df_job_quality_comp = df_2015_comp.groupby(['high_completion', 'group'])[measure_list].agg(['mean']).reset_index()

#See the result
df_job_quality_comp

In [ ]:
#Job Quality by major for non-completers and degree pursuers
df_job_quality_non_comp = df_2015_non_comp.groupby(['first_enroll', 'group'])[measure_list].agg(['mean']).reset_index()

#See the result
df_job_quality_non_comp

#### **Checkpoint 4: Examine Job Quality By Major**

1. Create two DataFrames. The first DataFrame only includes the 2013 cohort completers. The second DataFrame only includes the 2013 cohort non-completers and degree pursuers.

2. Examine job quality by majors for completers, non-completers, and degree pursuers in the 2013 cohort.

## 3. Export Results to csv Files

You can export data to csv files and save them in your personal folder. For example, we have generated:
- `df_cnt_group`: counts and percentages of the 2015 cohort who are completers, non-completers, and degree pursuers.
- `df_job_quality_group`: job quality measures for completers, non-completers, and degree pursuers.
- `df_job_quality_comp`: job quality measures by major for completers.
- `df_job_quality_non_comp`: job quality measures by major for non-completers and degree pursuers.

To export these results, we can use `.to_csv()`. In the parentheses, you need to define your directory and file name. The code `index = False` is to avoid exporting the index in each DataFrame.

> Before running the code below, please change <font color='red'> **YOUR USERNAME**</font> to your username or use your own file path.

In [ ]:
#Export Results to csv files
df_cnt_group.to_csv(r"C:\Users\YOUR USERNAME\Documents\cnt_group.csv", index = False)
df_job_quality_group.to_csv(r"C:\Users\YOUR USERNAME\Documents\job_quality_group.csv", index = False)
df_job_quality_comp.to_csv(r"C:\Users\YOUR USERNAME\Documents\job_quality_comp.csv", index = False)
df_job_quality_non_comp.to_csv(r"C:\Users\YOUR USERNAME\Documents\job_quality_non_comp.csv", index = False)